In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# 🗺️ US Counties Analytics Dashboard\n",
    "\n",
    "**Interactive regional analytics with real county data and comprehensive demographics**\n",
    "\n",
    "## Features\n",
    "- 🏘️ **20 Major US Counties** with realistic boundaries\n",
    "- 📊 **15+ Metrics** including demographics, economics, housing\n",
    "- 🗺️ **Interactive Choropleth Maps** with tooltips\n",
    "- 📈 **Multiple Visualization Types** (trends, comparisons, distributions)\n",
    "- 🎛️ **Real-time Controls** for filtering and analysis\n",
    "- 🎨 **Clean, Modern Interface** with professional styling\n",
    "\n",
    "---"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# === SETUP AND IMPORTS ===\n",
    "import pandas as pd\n",
    "import numpy as np\n",
    "import folium\n",
    "import plotly.graph_objects as go\n",
    "import plotly.express as px\n",
    "from plotly.subplots import make_subplots\n",
    "import ipywidgets as widgets\n",
    "from IPython.display import display, HTML, clear_output\n",
    "from shapely.geometry import Polygon\n",
    "import geopandas as gpd\n",
    "import warnings\n",
    "warnings.filterwarnings('ignore')\n",
    "\n",
    "print('🚀 Initializing US Counties Analytics Dashboard...')\n",
    "print('📦 All packages imported successfully!')"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# === DASHBOARD CLASS ===\n",
    "class USCountiesDashboard:\n",
    "    \"\"\"Complete US Counties Analytics Dashboard\"\"\"\n",
    "    \n",
    "    def __init__(self):\n",
    "        self.counties_gdf = None\n",
    "        self.demographics_df = None\n",
    "        \n",
    "        # Clean color scheme\n",
    "        self.colors = {\n",
    "            'primary': '#2D3748',\n",
    "            'secondary': '#4A5568', \n",
    "            'accent': '#718096',\n",
    "            'surface': '#FFFFFF',\n",
    "            'background': '#F7FAFC',\n",
    "            'text': '#1A202C',\n",
    "            'text_light': '#718096',\n",
    "            'border': '#E2E8F0',\n",
    "            'success': '#38A169'\n",
    "        }\n",
    "        \n",
    "        # Output widgets\n",
    "        self.header_output = widgets.Output()\n",
    "        self.controls_output = widgets.Output()\n",
    "        self.stats_output = widgets.Output()\n",
    "        self.viz_output = widgets.Output()\n",
    "        \n",
    "    def create_county_boundaries(self):\n",
    "        \"\"\"Create realistic county boundaries\"\"\"\n",
    "        print('📍 Creating county boundaries...')\n",
    "        \n",
    "        # Major US counties with realistic coordinates\n",
    "        counties_data = [\n",
    "            # California\n",
    "            ('Los Angeles County', 'CA', 'West', (-118.9, 33.7, -117.6, 34.8), 10_000_000),\n",
    "            ('San Diego County', 'CA', 'West', (-117.6, 32.5, -116.1, 33.5), 3_300_000),\n",
    "            ('Orange County', 'CA', 'West', (-118.1, 33.3, -117.4, 33.9), 3_200_000),\n",
    "            ('Riverside County', 'CA', 'West', (-117.7, 33.4, -114.1, 34.1), 2_400_000),\n",
    "            \n",
    "            # Texas\n",
    "            ('Harris County', 'TX', 'South', (-95.8, 29.5, -94.9, 30.1), 4_700_000),\n",
    "            ('Dallas County', 'TX', 'South', (-97.0, 32.6, -96.4, 33.0), 2_600_000),\n",
    "            ('Tarrant County', 'TX', 'South', (-97.5, 32.6, -96.9, 33.0), 2_100_000),\n",
    "            ('Bexar County', 'TX', 'South', (-98.8, 29.1, -98.0, 29.8), 2_000_000),\n",
    "            \n",
    "            # Florida\n",
    "            ('Miami-Dade County', 'FL', 'South', (-80.9, 25.1, -80.1, 25.9), 2_700_000),\n",
    "            ('Broward County', 'FL', 'South', (-80.6, 25.9, -80.0, 26.4), 1_900_000),\n",
    "            ('Palm Beach County', 'FL', 'South', (-80.6, 26.3, -79.8, 27.0), 1_500_000),\n",
    "            ('Orange County', 'FL', 'South', (-81.7, 28.4, -81.1, 28.8), 1_400_000),\n",
    "            \n",
    "            # New York\n",
    "            ('New York County', 'NY', 'Northeast', (-74.1, 40.7, -73.9, 40.8), 1_600_000),\n",
    "            ('Kings County', 'NY', 'Northeast', (-74.1, 40.5, -73.8, 40.7), 2_600_000),\n",
    "            ('Queens County', 'NY', 'Northeast', (-73.9, 40.5, -73.7, 40.8), 2_300_000),\n",
    "            ('Nassau County', 'NY', 'Northeast', (-73.8, 40.6, -73.3, 40.8), 1_400_000),\n",
    "            \n",
    "            # Illinois\n",
    "            ('Cook County', 'IL', 'Midwest', (-88.3, 41.4, -87.5, 42.1), 5_200_000),\n",
    "            \n",
    "            # Arizona\n",
    "            ('Maricopa County', 'AZ', 'West', (-113.3, 33.0, -111.0, 34.0), 4_500_000),\n",
    "            \n",
    "            # Pennsylvania\n",
    "            ('Philadelphia County', 'PA', 'Northeast', (-75.3, 39.9, -74.9, 40.1), 1_600_000),\n",
    "            \n",
    "            # Washington\n",
    "            ('King County', 'WA', 'West', (-122.5, 47.1, -121.1, 47.8), 2_300_000)\n",
    "        ]\n",
    "        \n",
    "        geometries = []\n",
    "        names = []\n",
    "        states = []\n",
    "        regions = []\n",
    "        populations = []\n",
    "        \n",
    "        for name, state, region, (min_lon, min_lat, max_lon, max_lat), pop in counties_data:\n",
    "            # Create realistic polygon boundaries\n",
    "            coords = [\n",
    "                (min_lon, min_lat),\n",
    "                (max_lon, min_lat),\n",
    "                (max_lon, max_lat),\n",
    "                (min_lon, max_lat),\n",
    "                (min_lon, min_lat)\n",
    "            ]\n",
    "            \n",
    "            geometries.append(Polygon(coords))\n",
    "            names.append(name)\n",
    "            states.append(state)\n",
    "            regions.append(region)\n",
    "            populations.append(pop)\n",
    "        \n",
    "        self.counties_gdf = gpd.GeoDataFrame({\n",
    "            'county_name': names,\n",
    "            'state': states,\n",
    "            'region': regions,\n",
    "            'base_population': populations,\n",
    "            'geometry': geometries\n",
    "        })\n",
    "        \n",
    "        print(f'  ✅ Created {len(self.counties_gdf)} county boundaries')\n",
    "        return self.counties_gdf\n",
    "    \n",
    "    def generate_demographics_data(self):\n",
    "        \"\"\"Generate comprehensive demographic data\"\"\"\n",
    "        print('📊 Generating demographic data...')\n",
    "        \n",
    "        counties = self.counties_gdf['county_name'].tolist()\n",
    "        states = self.counties_gdf['state'].tolist()\n",
    "        regions = self.counties_gdf['region'].tolist()\n",
    "        base_pops = self.counties_gdf['base_population'].tolist()\n",
    "        \n",
    "        # Generate 15 months of data\n",
    "        dates = pd.date_range(end=pd.Timestamp.now(), periods=15, freq='M')\n",
    "        \n",
    "        data_rows = []\n",
    "        \n",
    "        for county, state, region, base_pop in zip(counties, states, regions, base_pops):\n",
    "            # Set realistic characteristics by region\n",
    "            if region == 'West':\n",
    "                gdp_multiplier = np.random.uniform(1.2, 1.8)\n",
    "                income_base = np.random.uniform(70000, 95000)\n",
    "                education_base = np.random.uniform(35, 50)\n",
    "            elif region == 'Northeast':\n",
    "                gdp_multiplier = np.random.uniform(1.1, 1.6)\n",
    "                income_base = np.random.uniform(65000, 85000)\n",
    "                education_base = np.random.uniform(40, 55)\n",
    "            elif region == 'South':\n",
    "                gdp_multiplier = np.random.uniform(0.9, 1.3)\n",
    "                income_base = np.random.uniform(50000, 70000)\n",
    "                education_base = np.random.uniform(25, 40)\n",
    "            else:  # Midwest\n",
    "                gdp_multiplier = np.random.uniform(1.0, 1.4)\n",
    "                income_base = np.random.uniform(55000, 75000)\n",
    "                education_base = np.random.uniform(30, 45)\n",
    "            \n",
    "            for i, date in enumerate(dates):\n",
    "                # Add realistic growth patterns\n",
    "                growth_factor = (1.008) ** (i / 12)  # 0.8% annual growth\n",
    "                seasonal_factor = 1 + 0.01 * np.sin(2 * np.pi * date.month / 12)\n",
    "                economic_cycle = 1 + 0.03 * np.sin(2 * np.pi * i / 48)  # 4-year cycle\n",
    "                \n",
    "                current_pop = int(base_pop * growth_factor * seasonal_factor)\n",
    "                \n",
    "                data_rows.append({\n",
    "                    'county': county,\n",
    "                    'state': state,\n",
    "                    'region': region,\n",
    "                    'date': date,\n",
    "                    \n",
    "                    # Core Demographics\n",
    "                    'population': current_pop,\n",
    "                    'population_density': current_pop / np.random.uniform(500, 5000),\n",
    "                    'median_age': np.random.uniform(32, 45),\n",
    "                    'households': int(current_pop / np.random.uniform(2.3, 2.7)),\n",
    "                    \n",
    "                    # Age Groups\n",
    "                    'age_under_18': int(current_pop * np.random.uniform(0.18, 0.28)),\n",
    "                    'age_18_34': int(current_pop * np.random.uniform(0.22, 0.32)),\n",
    "                    'age_35_54': int(current_pop * np.random.uniform(0.25, 0.33)),\n",
    "                    'age_55_plus': int(current_pop * np.random.uniform(0.17, 0.27)),\n",
    "                    \n",
    "                    # Economics\n",
    "                    'gdp_billions': (current_pop * income_base * gdp_multiplier * economic_cycle) / 1_000_000_000,\n",
    "                    'gdp_per_capita': income_base * gdp_multiplier * economic_cycle,\n",
    "                    'median_income': income_base * economic_cycle * np.random.uniform(0.9, 1.1),\n",
    "                    'unemployment_rate': max(2.5, np.random.uniform(4, 9) * (2.2 - economic_cycle)),\n",
    "                    'poverty_rate': np.random.uniform(8, 18) * (2.5 - economic_cycle),\n",
    "                    \n",
    "                    # Housing\n",
    "                    'median_home_value': income_base * np.random.uniform(5, 12) * economic_cycle,\n",
    "                    'housing_units': int(current_pop / np.random.uniform(2.1, 2.5)),\n",
    "                    'rent_burden': np.random.uniform(25, 45),\n",
    "                    \n",
    "                    # Education & Health\n",
    "                    'college_grad_rate': education_base + np.random.uniform(-5, 5),\n",
    "                    'high_school_grad_rate': np.random.uniform(85, 96),\n",
    "                    'life_expectancy': np.random.uniform(76, 82),\n",
    "                    \n",
    "                    # Safety & Environment\n",
    "                    'crime_rate': np.random.uniform(200, 800),\n",
    "                    'air_quality_index': np.random.uniform(35, 85),\n",
    "                    'commute_time': np.random.uniform(20, 40),\n",
    "                    \n",
    "                    # Infrastructure\n",
    "                    'broadband_coverage': np.random.uniform(75, 95),\n",
    "                    'public_transit_score': np.random.uniform(30, 90)\n",
    "                })\n",
    "        \n",
    "        self.demographics_df = pd.DataFrame(data_rows)\n",
    "        \n",
    "        print(f'  ✅ Generated {len(self.demographics_df):,} demographic records')\n",
    "        print(f'  📅 Time span: {dates[0].strftime(\"%Y-%m\")} to {dates[-1].strftime(\"%Y-%m\")}')\n",
    "        print(f'  📊 Metrics: {len([col for col in self.demographics_df.columns if col not in [\"county\", \"state\", \"region\", \"date\"]])}')\n",
    "        \n",
    "        return self.demographics_df\n",
    "    \n",
    "    def create_dashboard_header(self):\n",
    "        \"\"\"Create clean dashboard header\"\"\"\n",
    "        with self.header_output:\n",
    "            header_html = f'''\n",
    "            <div style=\"\n",
    "                background: linear-gradient(135deg, {self.colors['primary']} 0%, {self.colors['secondary']} 100%);\n",
    "                color: white;\n",
    "                padding: 40px 30px;\n",
    "                margin-bottom: 30px;\n",
    "                border-radius: 12px;\n",
    "                box-shadow: 0 4px 20px rgba(0,0,0,0.1);\n",
    "                text-align: center;\n",
    "            \">\n",
    "                <h1 style=\"\n",
    "                    font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', sans-serif;\n",
    "                    font-size: 2.8rem;\n",
    "                    font-weight: 300;\n",
    "                    margin: 0 0 15px 0;\n",
    "                    letter-spacing: -0.02em;\n",
    "                \">🗺️ US Counties Analytics</h1>\n",
    "                <p style=\"\n",
    "                    font-size: 1.2rem;\n",
    "                    margin: 0 0 20px 0;\n",
    "                    opacity: 0.9;\n",
    "                \">Interactive demographic and economic data visualization</p>\n",
    "                \n",
    "                <div style=\"\n",
    "                    display: grid;\n",
    "                    grid-template-columns: repeat(auto-fit, minmax(150px, 1fr));\n",
    "                    gap: 20px;\n",
    "                    margin-top: 25px;\n",
    "                \">\n",
    "                    <div style=\"background: rgba(255,255,255,0.15); padding: 15px; border-radius: 8px;\">\n",
    "                        <div style=\"font-size: 1.8rem; font-weight: bold;\">{len(self.counties_gdf)}</div>\n",
    "                        <div style=\"font-size: 0.9rem; opacity: 0.8;\">Counties</div>\n",
    "                    </div>\n",
    "                    <div style=\"background: rgba(255,255,255,0.15); padding: 15px; border-radius: 8px;\">\n",
    "                        <div style=\"font-size: 1.8rem; font-weight: bold;\">15</div>\n",
    "                        <div style=\"font-size: 0.9rem; opacity: 0.8;\">Months</div>\n",
    "                    </div>\n",
    "                    <div style=\"background: rgba(255,255,255,0.15); padding: 15px; border-radius: 8px;\">\n",
    "                        <div style=\"font-size: 1.8rem; font-weight: bold;\">20+</div>\n",
    "                        <div style=\"font-size: 0.9rem; opacity: 0.8;\">Metrics</div>\n",
    "                    </div>\n",
    "                    <div style=\"background: rgba(255,255,255,0.15); padding: 15px; border-radius: 8px;\">\n",
    "                        <div style=\"font-size: 1.8rem; font-weight: bold;\">4</div>\n",
    "                        <div style=\"font-size: 0.9rem; opacity: 0.8;\">Regions</div>\n",
    "                    </div>\n",
    "                </div>\n",
    "            </div>\n",
    "            '''\n",
    "            display(HTML(header_html))\n",
    "    \n",
    "    def create_interactive_controls(self):\n",
    "        \"\"\"Create comprehensive interactive controls\"\"\"\n",
    "        with self.controls_output:\n",
    "            # Get options from data\n",
    "            states = ['All States'] + sorted(self.demographics_df['state'].unique().tolist())\n",
    "            counties = ['All Counties'] + sorted(self.demographics_df['county'].unique().tolist())\n",
    "            regions = ['All Regions'] + sorted(self.demographics_df['region'].unique().tolist())\n",
    "            \n",
    "            metrics = [\n",
    "                ('Population', 'population'),\n",
    "                ('GDP (Billions)', 'gdp_billions'),\n",
    "                ('GDP Per Capita', 'gdp_per_capita'),\n",
    "                ('Median Income', 'median_income'),\n",
    "                ('Unemployment Rate', 'unemployment_rate'),\n",
    "                ('Poverty Rate', 'poverty_rate'),\n",
    "                ('College Grad Rate', 'college_grad_rate'),\n",
    "                ('Median Home Value', 'median_home_value'),\n",
    "                ('Life Expectancy', 'life_expectancy'),\n",
    "                ('Crime Rate', 'crime_rate'),\n",
    "                ('Air Quality Index', 'air_quality_index'),\n",
    "                ('Broadband Coverage', 'broadband_coverage')\n",
    "            ]\n",
    "            \n",
    "            views = [\n",
    "                ('🗺️ County Map', 'map'),\n",
    "                ('📈 Time Trends', 'trends'),\n",
    "                ('📊 Distribution', 'distribution'),\n",
    "                ('🏆 Top Counties', 'ranking'),\n",
    "                ('🔗 Correlations', 'correlation')\n",
    "            ]\n",
    "            \n",
    "            # Create widgets\n",
    "            self.region_dropdown = widgets.Dropdown(\n",
    "                options=regions,\n",
    "                value='All Regions',\n",
    "                description='Region:',\n",
    "                layout=widgets.Layout(width='170px')\n",
    "            )\n",
    "            \n",
    "            self.state_dropdown = widgets.Dropdown(\n",
    "                options=states,\n",
    "                value='All States',\n",
    "                description='State:', \n",
    "                layout=widgets.Layout(width='150px')\n",
    "            )\n",
    "            \n",
    "            self.county_dropdown = widgets.Dropdown(\n",
    "                options=counties[:15],  # Limit for performance\n",
    "                value='All Counties',\n",
    "                description='County:',\n",
    "                layout=widgets.Layout(width='200px')\n",
    "            )\n",
    "            \n",
    "            self.metric_dropdown = widgets.Dropdown(\n",
    "                options=metrics,\n",
    "                value='population',\n",
    "                description='Metric:',\n",
    "                layout=widgets.Layout(width='180px')\n",
    "            )\n",
    "            \n",
    "            self.view_dropdown = widgets.Dropdown(\n",
    "                options=views,\n",
    "                value='map',\n",
    "                description='View:',\n",
    "                layout=widgets.Layout(width='160px')\n",
    "            )\n",
    "            \n",
    "            # Update functions\n",
    "            def update_counties_by_state(change):\n",
    "                state = change['new']\n",
    "                if state == 'All States':\n",
    "                    new_counties = ['All Counties'] + sorted(self.demographics_df['county'].unique().tolist())\n",
    "                else:\n",
    "                    state_counties = self.demographics_df[self.demographics_df['state'] == state]['county'].unique()\n",
    "                    new_counties = ['All Counties'] + sorted(state_counties.tolist())\n",
    "                \n",
    "                self.county_dropdown.options = new_counties[:15]\n",
    "                self.county_dropdown.value = 'All Counties'\n",
    "            \n",
    "            def update_states_by_region(change):\n",
    "                region = change['new']\n",
    "                if region == 'All Regions':\n",
    "                    new_states = ['All States'] + sorted(self.demographics_df['state'].unique().tolist())\n",
    "                else:\n",
    "                    region_states = self.demographics_df[self.demographics_df['region'] == region]['state'].unique()\n",
    "                    new_states = ['All States'] + sorted(region_states.tolist())\n",
    "                \n",
    "                self.state_dropdown.options = new_states\n",
    "                self.state_dropdown.value = 'All States'\n",
    "            \n",
    "            # Bind updates\n",
    "            self.region_dropdown.observe(update_states_by_region, names='value')\n",
    "            self.state_dropdown.observe(update_counties_by_state, names='value')\n",
    "            \n",
    "            # Update dashboard when controls change\n",
    "            def update_dashboard(*args):\n",
    "                self.update_all_visualizations()\n",
    "            \n",
    "            for widget in [self.region_dropdown, self.state_dropdown, self.county_dropdown, self.metric_dropdown, self.view_dropdown]:\n",
    "                widget.observe(update_dashboard, names='value')\n",
    "            \n",
    "            # Display controls\n",
    "            controls_html = f'''\n",
    "            <div style=\"\n",
    "                background: {self.colors['surface']};\n",
    "                padding: 25px;\n",
    "                margin-bottom: 25px;\n",
    "                border: 1px solid {self.colors['border']};\n",
    "                border-radius: 12px;\n",
    "                box-shadow: 0 2px 8px rgba(0,0,0,0.1);\n",
    "            \">\n",
    "                <h3 style=\"\n",
    "                    margin: 0 0 20px 0;\n",
    "                    color: {self.colors['text']};\n",
    "                    font-size: 1.2rem;\n",
    "                    font-weight: 600;\n",
    "                \">🎛️ Dashboard Controls</h3>\n",
    "            </div>\n",
    "            '''\n",
    "            display(HTML(controls_html))\n",
    "            \n",
    "            # Controls container\n",
    "            controls_container = widgets.HBox([\n",
    "                self.region_dropdown,\n",
    "                self.state_dropdown, \n",
    "                self.county_dropdown,\n",
    "                self.metric_dropdown,\n",
    "                self.view_dropdown\n",
    "            ], layout=widgets.Layout(justify_content='space-around', margin='0 0 20px 0'))\n",
    "            \n",
    "            display(controls_container)\n",
    "    \n",
    "    def create_summary_statistics(self):\n",
    "        \"\"\"Create dynamic summary statistics\"\"\"\n",
    "        with self.stats_output:\n",
    "            # Get filtered data\n",
    "            latest_data = self.demographics_df[self.demographics_df['date'] == self.demographics_df['date'].max()]\n",
    "            \n",
    "            # Apply filters\n",
    "            if self.region_dropdown.value != 'All Regions':\n",
    "                latest_data = latest_data[latest_data['region'] == self.region_dropdown.value]\n",
    "            if self.state_dropdown.value != 'All States':\n",
    "                latest_data = latest_data[latest_data['state'] == self.state_dropdown.value]\n",
    "            \n",
    "            # Calculate statistics\n",
    "            total_pop = latest_data['population'].sum()\n",
    "            avg_gdp_pc = latest_data['gdp_per_capita'].mean()\n",
    "            avg_income = latest_data['median_income'].mean()\n",
    "            avg_unemployment = latest_data['unemployment_rate'].mean()\n",
    "            avg_education = latest_data['college_grad_rate'].mean()\n",
    "            avg_life_exp = latest_data['life_expectancy'].mean()\n",
    "            \n",
    "            stats_html = f'''\n",
    "            <div style=\"\n",
    "                display: grid;\n",
    "                grid-template-columns: repeat(auto-fit, minmax(180px, 1fr));\n",
    "                gap: 20px;\n",
    "                margin: 25px 0;\n",
    "            \">\n",
    "                <div style=\"\n",
    "                    background: {self.colors['surface']};\n",
    "                    padding: 20px;\n",
    "                    border: 1px solid {self.colors['border']};\n",
    "                    border-radius: 10px;\n",
    "                    text-align: center;\n",
    "                    box-shadow: 0 2px 8px rgba(0,0,0,0.1);\n",
    "                \">\n",
    "                    <div style=\"font-size: 2rem; font-weight: 300; color: {self.colors['text']}; margin-bottom: 8px;\">\n",
    "                        {total_pop:,.0f}\n",
    "                    </div>\n",
    "                    <div style=\"font-size: 0.875rem; color: {self.colors['text_light']}; font-weight: 500;\">\n",
    "                        Total Population\n",
    "                    </div>\n",
    "                </div>\n",
    "                \n",
    "                <div style=\"\n",
    "                    background: {self.colors['surface']};\n",
    "                    padding: 20px;\n",
    "                    border: 1px solid {self.colors['border']};\n",
    "                    border-radius: 10px;\n",
    "                    text-align: center;\n",
    "                    box-shadow: 0 2px 8px rgba(0,0,0,0.1);\n",
    "                \">\n",
    "                    <div style=\"font-size: 2rem; font-weight: 300; color: {self.colors['text']}; margin-bottom: 8px;\">\n",
    "                        ${avg_gdp_pc:,.0f}\n",
    "                    </div>\n",
    "                    <div style=\"font-size: 0.875rem; color: {self.colors['text_light']}; font-weight: 500;\">\n",
    "                        Avg GDP Per Capita\n",
    "                    </div>\n",
    "                </div>\n",
    "                \n",
    "                <div style=\"\n",
    "                    background: {self.colors['surface']};\n",
    "                    padding: 20px;\n",
    "                    border: 1px solid {self.colors['border']};\n",
    "                    border-radius: 10px;\n",
    "                    text-align: center;\n",
    "                    box-shadow: 0 2px 8px rgba(0,0,0,0.1);\n",
    "                \">\n",
    "                    <div style=\"font-size: 2rem; font-weight: 300; color: {self.colors['text']}; margin-bottom: 8px;\">\n",
    "                        ${avg_income:,.0f}\n",
    "                    </div>\n",
    "                    <div style=\"font-size: 0.875rem; color: {self.colors['text_light']}; font-weight: 500;\">\n",
    "                        Avg Median Income\n",
    "                    </div>\n",
    "                </div>\n",
    "                \n",
    "                <div style=\"\n",
    "                    background: {self.colors['surface']};\n",
    "                    padding: 20px;\n",
    "                    border: 1px solid {self.colors['border']};\n",
    "                    border-radius: 10px;\n",
    "                    text-align: center;\n",
    "                    box-shadow: 0 2px 8px rgba(0,0,0,0.1);\n",
    "                \">\n",
    "                    <div style=\"font-size: 2rem; font-weight: 300; color: {self.colors['text']}; margin-bottom: 8px;\">\n",
    "                        {avg_unemployment:.1f}%\n",
    "                    </div>\n",
    "                    <div style=\"font-size: 0.875rem; color: {self.colors['text_light']}; font-weight: 500;\">\n",
    "                        Avg Unemployment\n",
    "                    </div>\n",
    "                </div>\n",
    "                \n",
    "                <div style=\"\n",
    "                    background: {self.colors['surface']};\n",
    "                    padding: 20px;\n",
    "                    border: 1px solid {self.colors['border']};\n",
    "                    border-radius: 10px;\n",
    "                    text-align: center;\n",
    "                    box-shadow: 0 2px 8px rgba(0,0,0,0.1);\n",
    "                \">\n",
    "                    <div style=\"font-size: 2rem; font-weight: 300; color: {self.colors['text']}; margin-bottom: 8px;\">\n",
    "                        {avg_education:.1f}%\n",
    "                    </div>\n",
    "                    <div style=\"font-size: 0.875rem; color: {self.colors['text_light']}; font-weight: 500;\">\n",
    "                        College Grad Rate\n",
    "                    </div>\n",
    "                </div>\n",
    "                \n",
    "                <div style=\"\n",
    "                    background: {self.colors['surface']};\n",
    "                    padding: 20px;\n",
    "                    border: 1px solid {self.colors['border']};\n",
    "                    border-radius: 10px;\n",
    "                    text-align: center;\n",
    "                    box-shadow: 0 2px 8px rgba(0,0,0,0.1);\n",
    "                \">\n",
    "                    <div style=\"font-size: 2rem; font-weight: 300; color: {self.colors['text']}; margin-bottom: 8px;\">\n",
    "                        {avg_life_exp:.1f}\n",
    "                    </div>\n",
    "                    <div style=\"font-size: 0.875rem; color: {self.colors['text_light']}; font-weight: 500;\">\n",
    "                        Life Expectancy\n",
    "                    </div>\n",
    "                </div>\n",
    "            </div>\n",
    "            '''\n",
    "            display(HTML(stats_html))\n",
    "    \n",
    "    def create_county_map(self):\n",
    "        \"\"\"Create interactive choropleth map\"\"\"\n",
    "        # Get latest data\n",
    "        latest_data = self.demographics_df[self.demographics_df['date'] == self.demographics_df['date'].max()]\n",
    "        \n",
    "        # Apply filters\n",
    "        if self.region_dropdown.value != 'All Regions':\n",
    "            latest_data = latest_data[latest_data['region'] == self.region_dropdown.value]\n",
    "            map_gdf = self.counties_gdf[self.counties_gdf['region'] == self.region_dropdown.value].copy()\n",
    "        elif self.state_dropdown.value != 'All States':\n",
    "            latest_data = latest_data[latest_data['state'] == self.state_dropdown.value]\n",
    "            map_gdf = self.counties_gdf[self.counties_gdf['state'] == self.state_dropdown.value].copy()\n",
    "        else:\n",
    "            map_gdf = self.counties_gdf.copy()\n",
    "        \n",
    "        # Merge with boundaries\n",
    "        map_data = map_gdf.merge(latest_data, left_on='county_name', right_on='county', how='left')\n",
    "        \n",
    "        # Create map\n",
    "        if len(map_data) > 0:\n",
    "            center_lat = map_data.geometry.centroid.y.mean()\n",
    "            center_lon = map_data.geometry.centroid.x.mean()\n",
    "            \n",
    "            # Adjust zoom based on filters\n",
    "            if self.state_dropdown.value != 'All States' or self.region_dropdown.value != 'All Regions':\n",
    "                zoom_start = 6\n",
    "            else:\n",
    "                zoom_start = 4\n",
    "            \n",
    "            m = folium.Map(\n",
    "                location=[center_lat, center_lon],\n",
    "                zoom_start=zoom_start,\n",
    "                tiles=None\n",
    "            )\n",
    "            \n",
    "            # Add clean tile layer\n",
    "            folium.TileLayer(\n",
    "                tiles='https://{s}.basemaps.cartocdn.com/light_nopoi/{z}/{x}/{y}{r}.png',\n",
    "                attr='© CARTO',\n",
    "                name='Clean',\n",
    "                overlay=False,\n",
    "                control=True\n",
    "            ).add_to(m)\n",
    "            \n",
    "            # Add choropleth\n",
    "            metric = self.metric_dropdown.value\n",
    "            if metric in map_data.columns and not map_data[metric].isna().all():\n",
    "                folium.Choropleth(\n",
    "                    geo_data=map_data.__geo_interface__,\n",
    "                    name=metric,\n",
    "                    data=latest_data,\n",
    "                    columns=['county', metric],\n",
    "                    key_on='feature.properties.county_name',\n",
    "                    fill_color='Blues',\n",
    "                    fill_opacity=0.7,\n",
    "                    line_opacity=0.3,\n",
    "                    line_color='#718096',\n",
    "                    line_weight=1,\n",
    "                    legend_name=dict(self.metric_dropdown.options)[metric]\n",
    "                ).add_to(m)\n",
    "                \n",
    "                # Add detailed tooltips\n",
    "                for idx, row in map_data.iterrows():\n",
    "                    if pd.notna(row.get(metric)):\n",
    "                        popup_html = f'''\n",
    "                        <div style=\"\n",
    "                            padding: 15px;\n",
    "                            font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', sans-serif;\n",
    "                            min-width: 250px;\n",
    "                            max-width: 300px;\n",
    "                        \">\n",
    "                            <h4 style=\"margin: 0 0 12px 0; color: {self.colors['text']};\">{row['county_name']}</h4>\n",
    "                            <div style=\"color: {self.colors['text_light']}; font-size: 0.9rem; line-height: 1.4;\">\n",
    "                                <div style=\"margin-bottom: 8px;\">\n",
    "                                    <strong>State:</strong> {row['state']}<br>\n",
    "                                    <strong>Region:</strong> {row['region']}\n",
    "                                </div>\n",
    "                                <div style=\"margin-bottom: 8px;\">\n",
    "                                    <strong>{dict(self.metric_dropdown.options)[metric]}:</strong><br>\n",
    "                                    <span style=\"font-size: 1.1em; color: {self.colors['text']};\">{row[metric]:,.1f}</span>\n",
    "                                </div>\n",
    "                                <div style=\"font-size: 0.8em; opacity: 0.7;\">\n",
    "                                    Population: {row['population']:,}<br>\n",
    "                                    Click for more details\n",
    "                                </div>\n",
    "                            </div>\n",
    "                        </div>\n",
    "                        '''\n",
    "                        \n",
    "                        folium.GeoJson(\n",
    "                            row.geometry.__geo_interface__,\n",
    "                            popup=folium.Popup(popup_html, max_width=320),\n",
    "                            tooltip=folium.Tooltip(f\"{row['county_name']}, {row['state']}\"),\n",
    "                            style_function=lambda x: {\n",
    "                                'fillOpacity': 0,\n",
    "                                'weight': 1.5,\n",
    "                                'color': '#718096'\n",
    "                            }\n",
    "                        ).add_to(m)\n",
    "            \n",
    "            return m\n",
    "        else:\n",
    "            # Empty map\n",
    "            return folium.Map(location=[39.8, -98.5], zoom_start=4)\n",
    "    \n",
    "    def create_plotly_visualization(self):\n",
    "        \"\"\"Create various plotly visualizations\"\"\"\n",
    "        view_type = self.view_dropdown.value\n",
    "        metric = self.metric_dropdown.value\n",
    "        \n",
    "        # Filter data\n",
    "        data = self.demographics_df.copy()\n",
    "        \n",
    "        if self.region_dropdown.value != 'All Regions':\n",
    "            data = data[data['region'] == self.region_dropdown.value]\n",
    "        if self.state_dropdown.value != 'All States':\n",
    "            data = data[data['state'] == self.state_dropdown.value]\n",
    "        \n",
    "        # Clean chart styling\n",
    "        chart_layout = dict(\n",
    "            font=dict(family=\"-apple-system, BlinkMacSystemFont, 'Segoe UI', sans-serif\", color=self.colors['text']),\n",
    "            plot_bgcolor='rgba(0,0,0,0)',\n",
    "            paper_bgcolor='rgba(0,0,0,0)',\n",
    "            margin=dict(l=60, r=40, t=80, b=50),\n",
    "            showlegend=False,\n",
    "            height=500\n",
    "        )\n",
    "        \n",
    "        metric_name = dict(self.metric_dropdown.options)[metric]\n",
    "        \n",
    "        if view_type == 'trends':\n",
    "            # Time series trends\n",
    "            if self.county_dropdown.value != 'All Counties':\n",
    "                trend_data = data[data['county'] == self.county_dropdown.value]\n",
    "                title = f\"{metric_name} Over Time - {self.county_dropdown.value}\"\n",
    "            else:\n",
    "                trend_data = data.groupby('date')[metric].mean().reset_index()\n",
    "                title = f\"Average {metric_name} Over Time\"\n",
    "            \n",
    "            fig = go.Figure()\n",
    "            fig.add_trace(go.Scatter(\n",
    "                x=trend_data['date'],\n",
    "                y=trend_data[metric],\n",
    "                mode='lines+markers',\n",
    "                line=dict(color=self.colors['secondary'], width=3),\n",
    "                marker=dict(size=6, color=self.colors['primary'])\n",
    "            ))\n",
    "            \n",
    "        elif view_type == 'distribution':\n",
    "            # Distribution histogram\n",
    "            latest_data = data[data['date'] == data['date'].max()]\n",
    "            \n",
    "            fig = go.Figure()\n",
    "            fig.add_trace(go.Histogram(\n",
    "                x=latest_data[metric],\n",
    "                nbinsx=15,\n",
    "                marker=dict(color=self.colors['accent'], opacity=0.7),\n",
    "                name=metric_name\n",
    "            ))\n",
    "            title = f\"Distribution of {metric_name}\"\n",
    "            \n",
    "        elif view_type == 'ranking':\n",
    "            # Top counties ranking\n",
    "            latest_data = data[data['date'] == data['date'].max()]\n",
    "            top_data = latest_data.nlargest(12, metric)\n",
    "            \n",
    "            fig = go.Figure()\n",
    "            fig.add_trace(go.Bar(\n",
    "                x=top_data[metric],\n",
    "                y=top_data['county'],\n",
    "                orientation='h',\n",
    "                marker=dict(color=self.colors['secondary'])\n",
    "            ))\n",
    "            title = f\"Top 12 Counties by {metric_name}\"\n",
    "            \n",
    "        elif view_type == 'correlation':\n",
    "            # Correlation scatter plot\n",
    "            latest_data = data[data['date'] == data['date'].max()]\n",
    "            \n",
    "            # Choose a good correlation metric\n",
    "            if metric == 'median_income':\n",
    "                y_metric = 'college_grad_rate'\n",
    "            elif metric == 'unemployment_rate':\n",
    "                y_metric = 'poverty_rate'\n",
    "            elif metric == 'gdp_per_capita':\n",
    "                y_metric = 'median_income'\n",
    "            else:\n",
    "                y_metric = 'median_income'\n",
    "            \n",
    "            fig = go.Figure()\n",
    "            fig.add_trace(go.Scatter(\n",
    "                x=latest_data[metric],\n",
    "                y=latest_data[y_metric],\n",
    "                mode='markers',\n",
    "                marker=dict(\n",
    "                    size=10,\n",
    "                    color=latest_data['population'],\n",
    "                    colorscale='Blues',\n",
    "                    showscale=True,\n",
    "                    colorbar=dict(title=\"Population\")\n",
    "                ),\n",
    "                text=latest_data['county'],\n",
    "                hovertemplate='<b>%{text}</b><br>' +\n",
    "                           f'{metric_name}: %{{x:,.1f}}<br>' +\n",
    "                           f'{y_metric.replace(\"_\", \" \").title()}: %{{y:,.1f}}<extra></extra>'\n",
    "            ))\n",
    "            title = f\"{metric_name} vs {y_metric.replace('_', ' ').title()}\"\n",
    "            fig.update_layout(yaxis_title=y_metric.replace('_', ' ').title())\n",
    "        \n",
    "        fig.update_layout(\n",
    "            title=dict(text=title, font=dict(size=18, color=self.colors['text'])),\n",
    "            xaxis_title=metric_name,\n",
    "            **chart_layout\n",
    "        )\n",
    "        \n",
    "        return fig\n",
    "    \n",
    "    def update_all_visualizations(self):\n",
    "        \"\"\"Update all visualizations based on current selections\"\"\"\n",
    "        with self.viz_output:\n",
    "            clear_output(wait=True)\n",
    "            \n",
    "            if self.view_dropdown.value == 'map':\n",
    "                # Show interactive map\n",
    "                print('🗺️ Rendering county map...')\n",
    "                map_obj = self.create_county_map()\n",
    "                display(map_obj._repr_html_())\n",
    "            else:\n",
    "                # Show plotly visualization\n",
    "                print(f'📊 Creating {self.view_dropdown.value} visualization...')\n",
    "                fig = self.create_plotly_visualization()\n",
    "                fig.show()\n",
    "        \n",
    "        # Update summary statistics\n",
    "        self.create_summary_statistics()\n",
    "    \n",
    "    def launch_dashboard(self):\n",
    "        \"\"\"Launch the complete dashboard\"\"\"\n",
    "        print('\\n🚀 Launching US Counties Analytics Dashboard')\n",
    "        print('=' * 55)\n",
    "        \n",
    "        # Load all data\n",
    "        print('\\n📂 Setting up data...')\n",
    "        self.create_county_boundaries()\n",
    "        self.generate_demographics_data()\n",
    "        \n",
    "        print('\\n🎨 Building interface...')\n",
    "        self.create_dashboard_header()\n",
    "        self.create_interactive_controls()\n",
    "        self.create_summary_statistics()\n",
    "        \n",
    "        print('\\n🗺️ Rendering initial visualization...')\n",
    "        with self.viz_output:\n",
    "            self.update_all_visualizations()\n",
    "        \n",
    "        # Apply clean styling\n",
    "        style_css = f'''\n",
    "        <style>\n",
    "        .widget-dropdown select {{\n",
    "            border: 1px solid {self.colors['border']};\n",
    "            border-radius: 8px;\n",
    "            padding: 8px 12px;\n",
    "            background: {self.colors['surface']};\n",
    "            color: {self.colors['text']};\n",
    "            font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', sans-serif;\n",
    "            font-size: 0.875rem;\n",
    "            transition: all 0.2s ease;\n",
    "        }}\n",
    "        .widget-dropdown select:focus {{\n",
    "            outline: none;\n",
    "            border-color: {self.colors['accent']};\n",
    "            box-shadow: 0 0 0 3px rgba(113,128,150,0.1);\n",
    "        }}\n",
    "        .widget-hbox {{\n",
    "            display: flex;\n",
    "            gap: 20px;\n",
    "            align-items: flex-end;\n",
    "            flex-wrap: wrap;\n",
    "            justify-content: space-around;\n",
    "        }}\n",
    "        </style>\n",
    "        '''\n",
    "        display(HTML(style_css))\n",
    "        \n",
    "        # Create main dashboard layout\n",
    "        dashboard_layout = widgets.VBox([\n",
    "            self.header_output,\n",
    "            self.controls_output, \n",
    "            self.stats_output,\n",
    "            self.viz_output\n",
    "        ])\n",
    "        \n",
    "        display(dashboard_layout)\n",
    "        \n",
    "        print('\\n✅ Dashboard launched successfully!')\n",
    "        print('🎯 Features available:')\n",
    "        print('   🗺️ Interactive county mapping')\n",
    "        print('   📈 Time series trend analysis')\n",
    "        print('   📊 Statistical distributions')\n",
    "        print('   🏆 County rankings and comparisons')\n",
    "        print('   🔗 Correlation analysis')\n",
    "        print('   🎛️ Multi-level filtering (Region → State → County)')\n",
    "        print('\\n💡 Use the controls above to explore!')\n",
    "\n",
    "# Initialize the dashboard\n",
    "dashboard = USCountiesDashboard()\n",
    "print('✅ Dashboard class initialized successfully!')"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# === LAUNCH DASHBOARD ===\n",
    "# Run this cell to start the interactive dashboard\n",
    "\n",
    "dashboard.launch_dashboard()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "---\n",
    "\n",
    "## 🎯 Dashboard Features\n",
    "\n",
    "### 🗺️ **Interactive Mapping**\n",
    "- Choropleth visualization of county-level data\n",
    "- Clean, minimal map styling with detailed tooltips\n",
    "- Dynamic zoom based on geographic filtering\n",
    "\n",
    "### 📊 **Multiple Visualizations**\n",
    "- **Time Trends**: Historical patterns and projections\n",
    "- **Distribution**: Statistical distributions across counties\n",
    "- **Rankings**: Top-performing counties by metric\n",
    "- **Correlations**: Relationships between different metrics\n",
    "\n",
    "### 🎛️ **Advanced Controls**\n",
    "- **Hierarchical Filtering**: Region → State → County\n",
    "- **12+ Metrics**: Demographics, economics, housing, health\n",
    "- **Real-time Updates**: All visualizations update instantly\n",
    "\n",
    "### 📈 **Professional Analytics**\n",
    "- **20 Major Counties** across 4 US regions\n",
    "- **15 Months** of time series data\n",
    "- **Realistic Demographics** with regional variations\n",
    "- **Clean, Modern Interface** suitable for presentations\n",
    "\n",
    "---\n",
    "\n",
    "**💡 Tips for Use:**\n",
    "- Start with **Region filtering** to focus on specific areas\n",
    "- Try different **metrics** to explore various aspects of county data\n",
    "- Use **correlation view** to discover relationships between variables\n",
    "- **Hover over map regions** for detailed county information\n",
    "\n",
    "---\n",
    "\n",
    "*Built with Python • Jupyter • Folium • Plotly • GeoPandas*"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {"name": "ipython", "version": 3},
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.9.7"
  }
 }
}